## Введение

Гуляя по парку, Зюк увидел ребят, играющих в футбол. Это странное, на первый взгляд, занятие, вызвало у него неподдельное любопытство. Мы предложили ему присоединиться, но Зюк смущённо отказался. Оказалось, что задача расчёта траектории полёта того или иного предмета считается у нейрян достаточно сложной и у Зюка до сих пор не было времени, чтобы реализовать нейросеть, которая могла бы эту задачу решить.

## Нейросеть

Мы с радостью вызвались помочь Зюку написать нейросеть для такого случая.

Вот что у нас получилось. Кое-где мы запутались и не смогли написать нужный кусок кода. Помогите нам, пожалуйста!

// внимательно изучите комментарии к каждой функции, <br>
// осознайте код настолько, насколько сможете. Заполните пропущенные кусочки. <br>
// за основу класса Network взят пример с сайта neuralnetworksanddeeplearning.com

In [2]:
import numpy as np
import random

def sigmoid(z):
    """The sigmoid function.
    Сигмоида
    """
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function.
    Производная сигмоиды по e (шутка). По x
    """
    return sigmoid(z)*(1-sigmoid(z))

def cost_function(network, test_data, onehot=True):
    c = 0
    for example, y in test_data:
        if not onehot:
            y = np.eye(3, 1, k=-int(y))
        yhat = network.feedforward(example)
        c += np.sum((y - yhat)**2)
    return c / len(test_data)

In [55]:
class Network:

    def __init__(self, sizes, biases, weights, output=True):
        """
        Список ``sizes`` содержит количество нейронов в соответствующих слоях
        нейронной сети. К примеру, если бы этот лист выглядел как [2, 3, 1],
        то мы бы получили трёхслойную нейросеть, с двумя нейронами в первом
        (входном), тремя нейронами во втором (промежуточном) и одним нейроном
        в третьем (выходном, внешнем) слое. Смещения и веса для нейронных сетей
        инициализируются случайными значениями, подчиняющимися стандартному нормальному
        распределению. Обратите внимание, что первый слой подразумевается слоем, 
        принимающим входные данные, поэтому мы не будем добавлять к нему смещение 
        (делать это не принято, поскольку смещения используются только при 
        вычислении выходных значений нейронов последующих слоёв)
        """

        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = biases
        self.weights = weights
        self.output = output

    def feedforward(self, a):
        """
        Вычислить и вернуть выходную активацию нейронной сети
        при получении ``a`` на входе (бывшее forward_pass).
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def backprop(self, x, y):
        """
        Возвращает кортеж ``(nabla_b, nabla_w)`` -- градиент целевой функции по всем параметрам сети.
        ``nabla_b`` и ``nabla_w`` -- послойные списки массивов ndarray,
        такие же, как self.biases и self.weights соответственно.
        """
        # Эту функцию необходимо реализовать
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # прямое распространение (forward pass)
        a =[0] * (self.num_layers + 1) # list for all activations
        z =[0] * (self.num_layers) # list for all z values
        
        
        for i, (b, w) in enumerate(zip(self.biases, self.weights)):
            if i == 0:
                print('input layer')
                print(f"x = {x}")
                activation = x
                a[0] = activation
                #print(f"activation = {activation}")
            #print(f'forward propagation number {i}')
            z[i] = np.dot(w, a[i]) + b
            a[i + 1] = sigmoid(z[i])
            #print(f"z^{i} = {z[i]}(output),   a\^{i} = {a[i+1]}(activation)")
            #activation = a[i + 1]

            

        # обратное распространение (backward pass)
        delta = sigmoid_prime(z[-1]) * self.cost_derivative(a[-1], y) # ошибка выходного слоя
        print(f"\delta^{self.num_layers} = {delta}")
        nabla_b[-1] = delta # производная J по смещениям выходного слоя
        print(f"dJ/db^{self.num_layers} = {delta}")
        print(f"a = {a}")
        nabla_w[-1] = np.dot(delta, a[-2].transpose()) # производная J по весам выходного слоя
        print(f"dJ/dw^{self.num_layers} = {nabla_w[-1]}")
        # Обратите внимание, что переменная l в цикле ниже используется
        # немного иначе, чем в лекциях.  Здесь l = 1 означает последний слой, 
        # l = 2 - предпоследний и так далее.  
        # Мы перенумеровали схему, чтобы с удобством для себя 
        # использовать тот факт, что в Python к переменной типа list 
        # можно обращаться по негативному индексу.
        for l in range(2, self.num_layers):
            # дополнительные вычисления, чтобы легче записывалось
            #
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sigmoid_prime(z[-l]) # ошибка на слое L-l
            print(f"\delta^{self.num_layers - l + 1} = {delta}")
            nabla_b[-l] = delta  # производная J по смещениям L-l-го слоя
            print(f"dJ/db^{self.num_layers - l + 1} = {delta}")
            nabla_w[-l] = np.dot(delta, a[-l-1].transpose())   # производная J по весам L-l-го слоя
            print(f"dJ/dw^{self.num_layers - l + 1} = {nabla_w[-l]}")    
        return nabla_b, nabla_w

    def evaluate(self, test_data):
        """
        Вернуть количество тестовых примеров, для которых нейронная сеть
        возвращает правильный ответ. Обратите внимание: подразумевается,
        что выход нейронной сети - это индекс, указывающий, какой из нейронов
        последнего слоя имеет наибольшую активацию.
        """
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """
        Возвращает вектор частных производных (\partial C_x) / (\partial a) 
        целевой функции по активациям выходного слоя.
        """
        return (output_activations-y)

In [56]:
sizes = [3, 2, 1]
biases = list(map(np.array,[[[0], [0]], [[0]]]))
weights = list(map(np.array,[[[2, 2, 2], [2, 2, 2]], [[1, 1]]])) 
network = Network(sizes=sizes, weights=weights, biases=biases)
#print(network.biases,'\n', network.weights)
network.backprop(x=np.array([[0],[1],[2]]), y=np.array(0))

input layer
x = [[0]
 [1]
 [2]]
\delta^3 = 0.0
dJ/db^3 = 0.0
a = [array([[0],
       [1],
       [2]]), array([[0.99752738],
       [0.99752738]]), array([[0.88027688]]), 0]
dJ/dw^3 = [[0.]]
\delta^2 = [[0.]
 [0.]]
dJ/db^2 = [[0.]
 [0.]]
dJ/dw^2 = [[0. 0.]
 [0. 0.]]


([array([[0.],
         [0.]]),
  0.0],
 [array([[0., 0.],
         [0., 0.]]),
  array([[0.]])])

In [57]:
# second network 
sizes = [3, 2, 1]
biases = list(map(np.array,[[[0], [0]], [[0]]]))
weights = list(map(np.array,[[[8, 7, 8], [10, 10, 9]], [[10, 9]]])) 
network = Network(sizes=sizes, weights=weights, biases=biases)
#print(network.biases,'\n', network.weights)
network.backprop(x=np.array([[0],[1],[2]]), y=np.array(1))

input layer
x = [[0]
 [1]
 [2]]
\delta^3 = -0.25
dJ/db^3 = -0.25
a = [array([[0],
       [1],
       [2]]), array([[1.],
       [1.]]), array([[0.99999999]]), 0]
dJ/dw^3 = [[-0.25]]
\delta^2 = [[-1.40069910e-08]
 [-1.26062919e-08]]
dJ/db^2 = [[-1.40069910e-08]
 [-1.26062919e-08]]
dJ/dw^2 = [[-1.40069910e-08 -1.40069910e-08]
 [-1.26062919e-08 -1.26062919e-08]]


([array([[-1.40069910e-08],
         [-1.26062919e-08]]),
  -0.25],
 [array([[-1.40069910e-08, -1.40069910e-08],
         [-1.26062919e-08, -1.26062919e-08]]),
  array([[-0.25]])])

In [58]:
# third network 
sizes = [3, 2, 1]
biases = list(map(np.array,[[[0], [0]], [[0]]]))
weights = list(map(np.array,[[[0.2, 0.9, 0.6], [0.2, 0.3, 0.7]], [[0.2, 0.5]]])) 
network = Network(sizes=sizes, weights=weights, biases=biases)
#print(network.biases,'\n', network.weights)
network.backprop(x=np.array([[15],[5],[15]]), y=np.array(1))

input layer
x = [[15]
 [ 5]
 [15]]
\delta^3 = -0.25
dJ/db^3 = -0.25
a = [array([[15],
       [ 5],
       [15]]), array([[0.99999993],
       [0.99999969]]), array([[0.66818774]]), 0]
dJ/dw^3 = [[-0.16704693]]
\delta^2 = [[-0.01108564]
 [-0.02771411]]
dJ/db^2 = [[-0.01108564]
 [-0.02771411]]
dJ/dw^2 = [[-0.01108564 -0.01108564]
 [-0.02771411 -0.0277141 ]]


([array([[-0.01108564],
         [-0.02771411]]),
  -0.25],
 [array([[-0.01108564, -0.01108564],
         [-0.02771411, -0.0277141 ]]),
  array([[-0.16704693]])])